In [ ]:
import pandas as pd
from collections import Counter

def map_chunk(df_chunk):
    helpful_reviews_counter = Counter()
    total_counter = Counter()

    for _, row in df_chunk.iterrows():
        key = (row["app_id"], row["app_name"])
        total_counter[key] += 1

        if row["votes_helpful"] >= 3:
            helpful_reviews_counter[key] += 1

    return helpful_reviews_counter, total_counter

def shuffle(mapped_counters):
    total_helpful = Counter()
    total_reviews = Counter()

    for helpful_counter, total_counter in mapped_counters:
        total_helpful.update(helpful_counter)
        total_reviews.update(total_counter)

    return total_helpful, total_reviews

def reduce(shuffled_helpful, shuffled_total):
    result = []

    for key in shuffled_total:
        helpful = shuffled_helpful.get(key, 0)
        total = shuffled_total[key]
        percentage = round((helpful / total) * 100, 2) if total > 0 else 0
        result.append((key, helpful, percentage))

    result.sort(key=lambda x: x[1], reverse=True)

    return result

def map_reduce(file, chunk_size=200_000):
    print(file)

    mapped = []
    handled_reviews = set()

    chunks = pd.read_csv(
        file,
        usecols = ["app_id", "app_name", "review_id", "votes_helpful"],
        dtype = {
            "app_id": "string",
            "app_name": "string",
            "review_id": "string",
            "votes_helpful": "Int64"
        },
        chunksize = chunk_size,
        # low_memory = True,
        engine = "c"
    )

    for chunk in chunks:
        chunk = chunk.dropna(subset=["review_id"])
        
        mask_new = ~chunk["review_id"].isin(handled_reviews)
        chunk_new = chunk.loc[mask_new]

        chunk_new = chunk_new.drop_duplicates(subset=["review_id"])

        if not chunk_new.empty:
            mapped.append(map_chunk(chunk_new))
            handled_reviews.update(chunk_new["review_id"].unique())

    shuffled_helpful, shuffled_total = shuffle(mapped)
    result = reduce(shuffled_helpful, shuffled_total)

    return result


file = r"../data/steam_reviews.csv"
result = map_reduce(file)

print("Статистика полезных отзывов у игр")
for (app_id, app_name), helpful_count, percentage in result:
    print(f"{app_name} (ID: {app_id}) - {helpful_count} ({percentage}%)")

./data/steam_reviews.csv
Статистика полезных отзывов у игр
PLAYERUNKNOWN'S BATTLEGROUNDS (ID: 578080) - 75241 (4.58%)
Grand Theft Auto V (ID: 271590) - 55543 (5.45%)
No Man's Sky (ID: 275850) - 42819 (23.52%)
PAYDAY 2 (ID: 218620) - 39084 (8.01%)
Fallout 4 (ID: 377160) - 37719 (16.47%)
ARK: Survival Evolved (ID: 346110) - 37594 (9.4%)
Tom Clancy's Rainbow Six Siege (ID: 359550) - 35486 (4.21%)
Rust (ID: 252490) - 28336 (5.16%)
Dead by Daylight (ID: 381210) - 27428 (6.55%)
The Witcher 3: Wild Hunt (ID: 292030) - 25383 (5.41%)
Garry's Mod (ID: 4000) - 23572 (3.6%)
Rocket League (ID: 252950) - 20632 (4.14%)
Terraria (ID: 105600) - 20367 (3.03%)
The Elder Scrolls V: Skyrim (ID: 72850) - 17589 (5.96%)
DARK SOULS™ III (ID: 374320) - 15544 (6.6%)
Euro Truck Simulator 2 (ID: 227300) - 14938 (3.85%)
The Forest (ID: 242760) - 14567 (6.08%)
NieR:Automata™ (ID: 524220) - 13947 (17.2%)
Sid Meier's Civilization VI (ID: 289070) - 13728 (9.55%)
The Elder Scrolls Online (ID: 306130) - 13465 (14.65%)
Ar